In [5]:
import requests
import xmltodict
import json
import folium
import pandas as pd 

# 공공데이터포털 사이트로 크롤링 요청 위한 인증키와 요청 메세지
key = "y3hJyZd4gVr4xFzmKzqb53nioC68jdCSKnf8lZYuhvp9VOgSJ5N1GgtPP7fEVhcasTygTCSvigiqKj2syoMEyg%3D%3D"
url_cycle="http://openapi.jejusi.go.kr/rest/PublicBikeInfoService/getPublicBikeInfoList?ServiceKey={}&pageNo=1&numOfRows=10".format(key)
url_parking = "http://openapi.jejuits.go.kr/OPEN_API/pisInfo/getPisInfo?serviceKey={}&pageNo=1&numOfRows=100".format(key)

# 웹에 요청
content_parking = requests.get(url_parking).content

# xml형식은 정보 분리하기 힘들어서 json으로 파싱
dict_parking = xmltodict.parse(content_parking)

#제주도맵 folium에 불러오기
jeju_map = folium.Map(location=[33.389398,126.541236], tiles='stamen Terrain',zoom_start=11)

# json으로 변화된 정보를 구분해서 출력-주차장
jsonString_parking = json.dumps(dict_parking['ServiceResult']['msgBody'], ensure_ascii = False)
jsonObj_parking = json.loads(jsonString_parking)
print('\n ----------------------------------데이터프레임으로 변환------------------------------')
df_park = pd.DataFrame(jsonObj_parking['itemList'])
df_park = df_park.loc[:,['ISTL_LCTN_ADDR','WHOL_NPLS','GNRL_RMND_PRZN_NUM']]
print(df_park) #데이터프레임형 리스트로 출력

# 지도에 위치를 마킹하기 위해 x,y좌표를 수동으로 입력해줌
df_p1 = pd.DataFrame({'x':[33.516518,33.504720,33.494673,33.494823,33.249802,33.512080,33.512090]}) # x좌표 lat
df_p2 = pd.DataFrame({'y':[126.525296,126.541789,126.535459,126.535294,126.562276,126.528340,126.528215]}) # y좌표 lon

#좌표를 리스트에 추가함
df_park = pd.concat([df_park,df_p1],axis=1)
df_park = pd.concat([df_park,df_p2],axis=1)
print(df_park) # 최종 리스트 출력

# for문을 반복하여 주차장의 위치를 지도에 마킹 + 설명과 잔여 주차가능 대수를 확인할수있음.
for x , y , name, num in zip(df_park.x , df_park.y , df_park.ISTL_LCTN_ADDR ,df_park.GNRL_RMND_PRZN_NUM):
    if int(num) < 10:
        folium.CircleMarker([x,y],
                         radius=4,
                         color='red',
                         fill=True,
                         fill_color='red',
                         fill_opacity=0.7,
                         popup=name+num
                       ).add_to(jeju_map)  # 10대 미만은 적색등.
        
    elif int(num) < 20:
          folium.CircleMarker([x,y],
                         radius=4,
                         color='orange',
                         fill=True,
                         fill_color='orange',
                         fill_opacity=0.7,
                         popup=name+num
                       ).add_to(jeju_map) # 10~20대 미만은 주황등.
            
    else :
        folium.CircleMarker([x,y],
                         radius=4,
                         color='green',
                         fill=True,
                         fill_color='green',
                         fill_opacity=0.7,
                         popup=name+num
                       ).add_to(jeju_map) # 20대 이상은 초록등.

        
#HTML 파일로 저장    
jeju_map.save('./jeju_park.html')       


 ----------------------------------데이터프레임으로 변환------------------------------
  ISTL_LCTN_ADDR WHOL_NPLS GNRL_RMND_PRZN_NUM
0            칠성골        80                 14
1           인제공영       125                 65
2           법원북측        91                 39
3           이도2동       150                 76
4         서귀중앙공영       306                  3
5      북수구(일도1동)        51                 38
6           동문시장       188                174
  ISTL_LCTN_ADDR WHOL_NPLS GNRL_RMND_PRZN_NUM          x           y
0            칠성골        80                 14  33.516518  126.525296
1           인제공영       125                 65  33.504720  126.541789
2           법원북측        91                 39  33.494673  126.535459
3           이도2동       150                 76  33.494823  126.535294
4         서귀중앙공영       306                  3  33.249802  126.562276
5      북수구(일도1동)        51                 38  33.512080  126.528340
6           동문시장       188                174  33.512090  126.528215
